Analysis of alpha-synuclein propagation
This code is used for the analysis of chloroquine propagation and oleic acid propagation experiments

In [1]:
# cell pose is machine learning model for segmenting cells
#from cellpose import models, utils, io

# for clearing jpynb output
from IPython.display import clear_output

# import necessary libraries
from scipy.ndimage import binary_fill_holes
from mpl_toolkits.mplot3d import Axes3D

import math
import skimage
import numpy as np
from czifile import CziFile
import czifile
from skimage import filters, morphology, segmentation
import numpy as np
from skimage.measure import regionprops
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.measure import label, regionprops
from skimage.io import imread
from skimage.measure import label
import czifile as czi
import pandas as pd
from skimage.filters import gaussian, threshold_otsu, sobel, median
from skimage.filters import threshold_otsu
from skimage import exposure
from skimage import morphology
from skimage.measure import label
from skimage.morphology import remove_small_objects
from skimage.morphology import binary_dilation, disk
from os import listdir
from os.path import isfile, join

In [5]:
# rework code into a function
def propagation_analysis(files, folder_path):
    '''
    propagation_analysis()
    Processes propogation czi files and measures the surface area of each channel (red and green) as well as the surface area of the intersection of red and not green
    inputs: list of file names, path to the file directory
    output: dataframe with image information and propogation measures 
    '''

    # initialize a mean sizes of inclusions list
    save_SAs_subtraction = []
    save_SAs_green = []
    removed_files = []
    row_lists = []

    # iterate through each image for analysis
    for path in files:
        
        # get path of image (this just ensures you are looking in the necessary folder since the images_to_analyze has just the file names)
        image_path = fr'{folder_path}\{path}'
        
        # get current image
        image = czifile.imread(image_path)
        image_squeezed = np.squeeze(image)

        # grab the red channel (DAPI)
        red_channel = image_squeezed[0,:,:]
        # Preprocess the image: Apply a Gaussian blur to the DAPI channel image to smooth out any noise. 
        blurred_red = gaussian(red_channel, sigma=2)
        # threhsold the red channel
        threshold_value = threshold_otsu(blurred_red)
        # create a binary mask over red channel
        binary_image = blurred_red > 0.1

        filled_image = morphology.remove_small_holes(binary_image, area_threshold=50000)
        # Remove objects with less than 100 pixels
        min_size = 400
        cleaned_image = remove_small_objects(filled_image, min_size=min_size) 
        # assume 'binary_image' is the binary image containing the segmented nuclei
        # set the size of the dilation element
        selem = disk(5)
        # dilate the binary image to merge adjacent objects - look at this
        merged_image = binary_dilation(cleaned_image, footprint=selem)
        # Label the objects: Use connected component analysis to label the segmented nuclei. 
        labeled_image = label(merged_image)

        fig, axes = plt.subplots(1, 3, figsize=(15, 9))
        # Plot each frame
        axes[0].imshow(red_channel)
        axes[0].set_title('red channel')
        axes[0].axis('off')

        axes[1].imshow(binary_image)
        axes[1].set_title("red channel binary image")
        axes[1].axis('off')

        axes[2].imshow(labeled_image, cmap='seismic')
        axes[2].set_title("merged and labeled red channel")
        axes[2].axis('off')
        plt.show()
        
        # extract the green channel
        green_channel = image_squeezed[1,:,:]     
        # normalize the green channel
        green_channel = (green_channel - green_channel.min()) / (green_channel.max() - green_channel.min()) 
        # threshold the image
        threshold_value = 0.05  # adjust this value as needed
        binary_image2 = green_channel > threshold_value
        # dilate the mask to connect closely related objects
        radius = 9
        selem = disk(radius)
        binary_image2_dilated = binary_dilation(binary_image2, selem)

        # Label the objects: Use connected component analysis to label the segmented inclusions. 
        labeled_image2 = label(binary_image2_dilated)

        fig, axes = plt.subplots(1, 3, figsize=(15, 9))
        # Plot each frame
        axes[0].imshow(green_channel)
        axes[0].set_title('green channel')
        axes[0].axis('off')

        axes[1].imshow(binary_image2)
        axes[1].set_title("thresholded image green channel")
        axes[1].axis('off')

        axes[2].imshow(labeled_image2, cmap='seismic')
        axes[2].set_title("thresholded image green channel dilated")
        axes[2].axis('off')
        plt.show()

        # Measure the size of each inclusion
        props_green = regionprops(labeled_image2) 
        # subtract off the gren fluorescence from the red fluorescnece, to single out areas where it is just red
        subtraction_mask = binary_image & ~binary_image2_dilated

        labeled_image_subtraction = label(subtraction_mask)

        plt.imshow(labeled_image_subtraction)
        plt.title("labeled image subtraction")
        plt.show()

        # take properties of each of each object in the subtracted image
        props_subtraction = regionprops(labeled_image_subtraction)

        # create a list to store the sizes of each individual inclusion
        
        # sum the surface area of each neuron (labeled object)
        SA_subtraction = 0
        # iterate over the properties of each subtracted labeled region (red without green)
        for prop in props_subtraction:
            # append the area of the region to the 'sizes' list
            SA_subtraction = SA_subtraction+prop.area

        save_SAs_subtraction.append(SA_subtraction)
        SA_green = 0
        for prop in props_green:
            # append the area of the region to the 'sizes_nuclei' list
            SA_green = SA_green+prop.area
        save_SAs_green.append(SA_green)

        # compute the ratio of red flourescnece without green fluorescence to green fluorescence (neurons accepting Syn but not producing it)
        ratio = SA_subtraction/SA_green
        print('ratio', SA_subtraction/SA_green)

        # Jiya's chunk for visually inspecting each image as it is printed
        add_df = input("Do you want to add this dataframe? (yes/no): ").lower()
        plt.close('all')
        plt.clf()
        clear_output(wait=True)
        if 'y' in add_df:
         #if basename in result_dict and i in result_dict[basename]:
         # if segmetnation is good, add the results to be returned at the end
           row_lists.append([path, SA_green, SA_subtraction, ratio])
        elif 'n' in add_df:
            removed_files.append(path)
            #with open(r"C:\Users\bs1250\Box\LAB\Lab Folder\WGCNA_Ben\Analysis\discarded_images_06192024.txt", 'w') as f:
            #    f.write(f"Discarded: {file}")
            #f.close()
        elif add_df == 'stop':
            raise Exception('stop input')

        row_lists.append([path, SA_green, SA_subtraction, ratio])

    # print the names of the removed files
    print('REMOVED FILES')
    for file in removed_files:
        print(file)

    results_df = pd.DataFrame(data=row_lists, columns=['image', 'SA_green', 'SA_subtraction', 'subtraction_green_ratio'])
    return results_df


In [ ]:
# Oleic acid propagation
oa_prop_folder = "oa_propagation_images"
oa_images_to_analyze = [f for f in listdir(oa_prop_folder) if isfile(join(oa_prop_folder, f))]
oa_results = propagation_analysis(oa_images_to_analyze, oa_prop_folder)
display(oa_results)



In [ ]:
# Chloroquine propagation
ch_prop_folder = "ch_propagation_images"
ch_images_to_analyze = [f for f in listdir(ch_prop_folder) if isfile(join(ch_prop_folder, f))]
ch_results = propagation_analysis(ch_images_to_analyze, ch_prop_folder)
display(ch_results)

